In [5]:
import pandas as pd
import requests
import time

# -------- CONFIG --------
INPUT_FILE = "t20_csv_files/dim_players_no_images.csv"
OUTPUT_FILE = "dim_players_with_images.csv"
PLAYER_COL = "name"      # change if your column name is different
IMAGE_COL = "image_url"
DEFAULT_IMAGE = "https://ifttt.com/images/no_image_card.png"
# ------------------------


def get_player_image(player_name):
    try:
        search_url = "https://www.wikidata.org/w/api.php"
        params = {
            "action": "wbsearchentities",
            "search": player_name,
            "language": "en",
            "format": "json",
            "limit": 1
        }

        r = requests.get(search_url, params=params, timeout=10)
        data = r.json()

        if not data.get("search"):
            return DEFAULT_IMAGE

        entity_id = data["search"][0]["id"]

        entity_url = f"https://www.wikidata.org/wiki/Special:EntityData/{entity_id}.json"
        entity_data = requests.get(entity_url, timeout=10).json()

        image_name = (
            entity_data["entities"][entity_id]["claims"]["P18"][0]
            ["mainsnak"]["datavalue"]["value"]
        )

        image_url = (
            "https://commons.wikimedia.org/wiki/Special:FilePath/"
            + image_name.replace(" ", "_")
        )

        return image_url

    except Exception:
        return DEFAULT_IMAGE


# Load CSV
df = pd.read_csv(INPUT_FILE)

# Create image column
df[IMAGE_COL] = df[PLAYER_COL].apply(get_player_image)

# Save CSV
df.to_csv(OUTPUT_FILE, index=False)

print("✅ Image URLs added successfully!")
print(f"📁 Output file: {OUTPUT_FILE}")



✅ Image URLs added successfully!
📁 Output file: dim_players_with_images.csv
